# Tarea - Semanal

## Dataset: Correos de Enron

Enron fue una empresa petrolera norteamericana que quebró estrepitosamente en el año 2001 luego de haber hecho lo que fue hasta su momento el fraude contable más grande de la historia. Como parte de la investigación criminal, se analizaron cientos de miles de mensajes de los servidores de correo electrónico de la compañía. 

El dataset está disponible para el público desde hace varios años. Aquí tenemos una muestra preprocesada con aproximadamente 10,000 mensajes de correo electrónico listos para ser analizados. 

In [13]:
# Importamos bibliotecas necesarias para la tarea
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from hmmlearn import hmm
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Leemos el dataset
enron_file = open('resources/enron.csv', 'r') 
enron = enron_file.readlines()
enron = [ l.rstrip() for l in enron ]

In [17]:
### EJERCICIO 1 (1 punto)
### - CONSTRUIR UN VECTORIZADOR DE CONTEOS QUE ELIMINE STOP-WORDS EN INGLÉS PARA EL DATASET enron
### - UTILIZARLO PARA TRANSFORMAR EL DATASET EN UNA MATRIZ DOCUMENTO-PALABA

cv = CountVectorizer(stop_words='english', token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z][a-zA-Z]+\b").fit(enron)

### TODO: Crear el count vectorizer y el fit.
enron_matrix = cv.transform(enron)

In [18]:
### EJERCICIO 2 (4 puntos)
### - ENTRENAR UN MODELO DE ASIGNACION LATENTE DE DIRICHLET SOBRE enron_matrix.
### - MOSTRAR LOS TÓPICOS RESULTANTES (las 30 palabras más relevantes por tópico)
### - PROBAR CON DISTINTOS NÚMEROS DE TÓPICOS, REPORTAR SUS RESULTADOS. 
from sklearn.decomposition import LatentDirichletAllocation

## Entrenamos el modelo LDA (toma un par de minutos)
lda = LatentDirichletAllocation(n_components=20, verbose=True).fit(enron_matrix)

comp = lda.components_
vec = np.array(cv.get_feature_names())

### EJERCICIO 2.1 (2 puntos)
### Despliegue las distribuciones de los topicos
### TODO: Desplegar los conteos de las distribuciones (ver notebook #2)
for i in range(0, comp.shape[0]):
    print('*** TOPICO ', i)
    print(vec[comp[i].argsort()[-30:][::-1]])
    print('\n')


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
*** TOPICO  0
['market' 'iso' 'power' 'energy' 'board' 'price' 'gas' 'california' 'ufe'
 'prices' 'said' 'electricity' 'trade' 'cap' 'state' 'san' 'data' 'new'
 'years' 'rates' 'think' 'going' 'high' 'just' 'project' 'dont' 'curve'
 'bid' 'time' 'week']


*** TOPICO  1
['week' 'play' 'sore' 'expected' 'start' 'saturday' 'phillip' 'update'
 'lucy' 'questionable' 'practiced' 'practice' 'missed' 'like' 'rent'
 'rentroll' 'likely' 'email' 'need' 'wednesday' 'sunday' 'did' 'news'
 'friday' 'paid' 'season' 'thursday' 'going' 'time' 'new']


*** TOPICO  2
['game' 'like' 'texas' 'brown' 'team' 'hes' 'players' 'good' 'said' 'week'
 'moving' 'think' 'season' 'just' 'yards' 'year' 'bowl' 'make' 'ball'
 'sunday'

In [22]:
### EJERCICIO 3 (1 punto)
### - CONSTRUIR UN LabelEncoder CON LOS DATOS DE enron Y TRANSFORMAR LOS DATOS
###   EL LabelEncoder DEBE SEPARAR LOS MENSAJES PALABRA POR PALABRA
### - TRANSFORMAR EN DATASETS CODIFICADOS
### - CREAR EL VECTOR DE LONGITUD DE MENSAJES
###
### *** IMPORTANTE: DADO EL TIEMPO EXCESIVO QUE TOMA ENTRENAR ESTE MODELO, 
###     SE RECOMIENDA TRABAJAR CON UNA MUESTRA REDUCIDA (p.ej. LOS PRIMEROS X MENSAJES)

enron_chars = [ mail.lower().split(' ') for mail in enron[-200:] ]

chars_len = [len(y) for y in enron_chars] ### TODO: crear arreglo de longitudes de los enron_chars

### TODO: realizar el LabelEncoder fir de enron_chars (Pista buscar enc en Notebook #3)

cv_chars = LabelEncoder().fit([y for l in enron_chars for y in l])

enron_enc = np.concatenate([cv_chars.transform(y).reshape(-1, 1) for y in enron_chars]).astype(np.int64)

In [23]:
### EJERCICIO 4 (4 puntos)
### - ENTRENAR UN MODELO OCULTO DE MARKOV (HMM) SOBRE chars_enc Y chars_len
### - GENERAR DIEZ SECUENCIAS ALEATORIAS DE DISTINTAS LONGITUDES
### - PROBAR CON DISTINTOS NÚMEROS DE COMPONENTES, REPORTAR SUS RESULTADOS. 

enron_model = hmm.MultinomialHMM(n_iter=10, n_components=10, verbose=True).fit(enron_enc, chars_len)

chars_enc = np.concatenate([cv_chars.transform(y).reshape(-1, 1) for y in enron_chars]).astype(np.int64)

Fitting a model with 82989 free scalar parameters with only 61870 data points will result in a degenerate solution.
         1     -558328.1704             +nan
         2     -441037.8675     +117290.3028
         3     -441031.7286          +6.1389
         4     -441022.4028          +9.3258
         5     -441008.6420         +13.7608
         6     -440990.7912         +17.8508
         7     -440970.4198         +20.3714
         8     -440948.3700         +22.0498
         9     -440924.2891         +24.0810
        10     -440897.6887         +26.6004


In [26]:
# Generar Cadenas aleatorias...
print(' '.join(cv_chars.inverse_transform(enron_model.sample(10)[0])))

can look blake and that way hector are practiced to


### Tarea Extra (no cuenta como entregable semanal):

En esta tarea ud va replicar este notebook pero con un texto mas largo y va a buscar entrenar el modelo con HMM para generar frases aleatorias.

Para este ejercicio va a a tratar de entrenar el modelo usando el libro de Dracula. https://www.gutenberg.org/files/345/345-0.txt (puede utilizar otros libros de us gusto, mas textos en https://www.gutenberg.org)

Este ejercicio requiere que ud realice un limapiado del texto primero para resultados optimos.


<img src="https://e7.pngegg.com/pngimages/518/88/png-clipart-dracula-from-sesame-street-count-von-count-elmo-sesame-street-characters-count-dracula-vampire-sesame-miscellaneous-purple-thumbnail.png" />
